## Summary 2020/08 - Thesis of Ning Yue - Goal: Implementation a useful agent in scml

### Second step - Implement a Deep Reinforcement Learning Negotiator in Negmas Simulator

Firstly, Yasser and Tim, Sorry for this too later Summary.

Due to too many reasons, the work are going too slowly. 

### Background of this Summary

This Summary obtains mainly two parts work.

1. The preparation
    - Read the paper, get the theory knowledge
    - Try to implemet two relative project.
2. Implement a special Environment for developing a DRL-Negotiator based on Gym and Negmas
    - Implement a Negotiation Environment
    - Implement a Negotiation Game
    - Implemet a DRL-Negotiator 
    - Evaluate a result

My idea is: Before i give you Summary, i need to get some readable result.

So after i have already read many relative papers and projects, i did not send you email immediately.

I try to understand these by myself. But naturally i have a lot of questions. 

So in this Summary i have wrote two parts: preparation(Normally i need to send to you last month) 
and Implement a special Environment based on Gym and Negmas for developing a DRL-Negotiator 

### Prepration

#### Papers and Theory Knowledge

1. (basic) from all of the papers that received from you, i got the basic Knowledge of negotiation, for example: issue, outcome, utiltiy function, negotiation mechanism, heuristic negotiation, pareto optimal,  The impact of time, Opponents model, Accepting Optimally, Alternating Offers Protocols and so on.

2. I have searched some papers by myself. for example
    - [Decoupling Negotiating Agents to Explore the Space of Negotiation Strategy](https://www.researchgate.net/publication/263157273)
    - [RLBOA: A Modular Reinforcement Learning Framework for Autonomous Negotiating Agents](https://homepages.cwi.nl/~baarslag/pub/RLBOA-A_Modular_Reinforcement_Learning_Framework_for_Autonomous_Negotiating_Agents.pdf)

        Above two papers are about Decoupling negotiation, implemented in [Genius](http://users.umiacs.umd.edu/~sarit/data/articles//linetal-CI.pdf), bidding strategy(propose in negmas), opponent model(), acceptance strategy(negmas.modeling.acceptance), all of these modul could be learned by Reinforcement learning.

    - [Concurrent bilateral negotiation for open e-markets:the CONAN strategy](https://doi.org/10.1007/s10115-017-1125-2)
    - [A Deep Reinforcement Learning Approach to Concurrent Bilateral Negotiation](https://www.researchgate.net/publication/338989771)
    
        From above two papers, i know, it is possible to use DRL to learn negotiation strategy in concurrent bilateral negotiation, also in simple bilateral negotiation using DRL is naturally possible. I refer to the ideas in these papers: observation space, reward, and action space
    
    - [Optimal Negotiation Strategies for Agents with Incomplete Information](https://www.researchgate.net/publication/221454311_Optimal_Negotiation_Strategies_for_Agents_with_Incomplete_Information)

        From this paper can get the Negotiation Environments, and about impact of time in negotiation (Linear, Conceder, Boulware) and so on. Negotiation Environments is useful for definition the observation space in reinfocement learning. later i will check it. 
        
    - [Multi-issue Bargaining with Deep Reinforcement Learning](https://arxiv.org/abs/2002.07788)
        
        I am reading it now, have not finished. I'm not sure if it's necessary. But i am sure, that is useful.
    And some papers about multi Agents, reinfocement learning, machine learning.

#### Two Project

##### [RLBOA](https://homepages.cwi.nl/~baarslag/pub/RLBOA-A_Modular_Reinforcement_Learning_Framework_for_Autonomous_Negotiating_Agents.pdf) in negmas

I have almost finished it. But not useful for my thesis, so when i know the basic framework, I have stopped working. I just want to learn how to write rl in decoupling negotiation framework, so if you need it, I can send to you.

I have also found similar modul in negmas, Advanced Negotiation.

##### [ANEGMA](https://www.researchgate.net/publication/338989771) in negmas

A drl negotiator in concurrent bilatreal negotiation, firstly, i want to realize it in negmas, but spent too much time, so i have just extracted some useful information from it. 


### Implement a special Environment based on Gym and Negmas for developing a DRL-Negotiator 

#### Basic Setting

SAOMechanism, All Negotiators are the sub-class of SAONegotiator

#### Components

1. DRL-Negotiator: MyNegotiator
2. Negotiation Game: NegotiationGame
3. Negotiation Env: AnegmaEnv

In [ ]:
!pip install negmas==0.7.0
!pip install gym==0.17.2
# Before install stable_baselines Need to install libopenmpi-dev: command 'sudo apt install libopenmpi-dev'
!pip install mpi4py==3.0.3
!pip install stable_baselines==2.10.0
!pip install tensorflow==1.15.3

#### Implemet a DRL-Negotiator

The main tasks of DRL Negotiator are as follows:

1. Received the observed state after every step, and then send to DRL Algorithm
2. Received the action decided by DRL Algorithm, and then send to Negmas Simulator


In [ ]:
'''
DRL-Negotiator!
'''

import numpy as np
from typing import Optional
from negmas import Action #  An action that an `Agent` can execute in a `World` through the `Simulator`
from negmas import (
                        Issue, 
                        MechanismState, # The mechanism state
                        SAOMechanism, 
                        SAONegotiator,  
                        MappingUtilityFunction, 
                        AspirationNegotiator,
                        ResponseType,
                    )
    
import random


def my_utility_function(offer, time, rp, ip, t_end):
    '''
        my utility function, 
        consider both the outcome and time
    '''
    d_t = 0.6
    # print(offer, time, ip, rp, t_end)
    # print(((float(rp) - float(offer[0])) / (float(rp) - float(ip))) * (float(time) / float(t_end)) ** d_t)
    return ((float(rp) - float(offer[0])) / (float(rp) - float(ip))) * (float(time) / float(t_end)) ** d_t

class MyNegotiator(SAONegotiator):
    
    def __init__(self):
        super().__init__(name="my_negotiator")
        self.current_action: Optional[ResponseType] = None
        self.end_time = 1
        self.initial_price = random.sample(range(300, 350), 1)[0]
        self.reserved_price = random.sample(range(500, 550), 1)[0]
        self.rational_proposal = True
        self.random_proposal = False
        # self.reserved_value = None
   
    def propose_(self, state: MechanismState) -> Optional["Outcome"]:
        
        if not self._ami._mechanism._running:
            return None
        proposal = self.propose(state=state)
        # import pdb;pdb.set_trace()
        # never return a proposal that is greater than the reserved value
        if self.rational_proposal:
            utility = None
            if proposal is not None and self._utility_function is not None:
                utility = self.get_utility(proposal)
                if utility is not None and utility < self.get_utility((self.reserved_price,)):
                    print('utility is less than the utility of reserved price')
                    # when the utility of proposaled offer is less than the utility of reserved price,
                    # then this proposaled offer is useless
                    self.checked_proposal = False
                    return None

            if utility is not None:
                self.my_last_proposal = proposal
                self._my_last_proposal = proposal 
                self.my_last_proposal_utility = utility
                self._my_last_proposal_utility = utility
        
        return proposal
        
    def respond(self, state: MechanismState, offer: "Outcome") -> "ResponseType":
        print("The response of my negotiator has been called")
        # if self.current_action == ResponseType.ACCEPT_OFFER:
        #     import pdb;pdb.set_trace()
        # print(self.current_action)
        # if self.current_action == ResponseType.ACCEPT_OFFER:
        #     import pdb;pdb.set_trace()

        if isinstance(self.current_action, ResponseType):
            return self.current_action
    
    def propose(self, state: MechanismState) -> Optional["Outcome"]:
        '''
            Here define the function that how to propose,
            just when the RepsonseType is REJECT_OFFER or the 
            offer from opponent is None, this function will be called!
        '''
        print("The propose of my negotiator has been called")
        print(self.current_action)
        # self.checked_proposal = False
        if isinstance(self.current_action, ResponseType) \
                and self.current_action == ResponseType.REJECT_OFFER:
            def _propose():
                '''
                    TODO: calculate the new outcome
                    initial: random propose, 
                    Improvement: the best outcome that has the best utility in current time
                '''
                if self.random_proposal:
                    return self._ami.outcomes[random.randint(0, len(self._ami.outcomes))-1]
                else:
                    # TODO: Continuous action space
                    # add one value, this is a Continuous action space, return Gaussian probability distribution,
                    # and then sample a value from this disctribution DDPG
                    # return self._ami.outcomes[random.randint(0, len(self._ami.outcomes))-1]
                    if self.my_last_proposal:
                        print('my last proposal has been called!', self.my_last_proposal[0] + 5)
                        return self.my_last_proposal[0] + 5, 
                    else:
                        print('initial price')
                        return self.initial_price + 1,
            
            # print("_propose called")
            self.checked_proposal = True 
            return _propose()
        else:
            # not offer received from opponent, so create random offer.
            self.checked_proposal = True
            return self._ami.outcomes[random.randint(0, len(self._ami.outcomes))-1]
    
    def get_last_proposal(self):
        return self.my_last_proposal

    def get_current_action(self):
        return self.current_action
        
    def set_current_action(self, action: int):
        # import pdb;pdb.set_trace()
        # print("set current action called", ResponseType(action))
        self.current_action = ResponseType(action)
    
    def get_utility(self, offer: Optional["outcome"]):
        return self.ufun(offer, self.get_time(), self.get_reserved_price(), \
                    self.get_initial_price(), self.get_maximum_end_time())
        
    def get_maximum_end_time(self):
        '''
            Get the end time of my negotiator
        '''
        if hasattr(self, 'end_time'):
            return self.end_time
        else:
            return float("inf")

    def get_initial_price(self):
        '''
            initial price which my negotiator offer
        '''
        return self.initial_price
        
    def get_reserved_price(self):
        '''
            Maximum price which my negotiator can offer to opponent
        '''
        # import pdb;pdb.set_trace()
        return self.reserved_price
    
    def get_time(self):
        # import pdb;pdb.set_trace()
        # print(self._ami._mechanism.id)
        # print(self._ami._mechanism.time)
        return self._ami._mechanism.time
    
    def get_obs(self, x_best) -> np.array:
        '''
        >>> get_obs(550)
        np.array([550, 150.0, 335, 522])
        '''
        x_best = x_best
        # import pdb;pdb.set_trace()
        time_left = self.get_maximum_end_time() - self.get_time()
        ip_my = self.get_initial_price()
        rp_my = self.get_reserved_price()
        
        result = [x_best, time_left, ip_my, rp_my]
        scale_factory =  1 # scale inputs to be in the order of magnitude of 100 for neural network
        return np.array(result) / scale_factory

#### Negotiation Game

Need to realize a Negotiation Game based on Negmas, The following functions are needed to realize in this Game:

1. The logic of game, manager the negotiator
2. base on the result of every step to define the reward, very important

In [ ]:
'''
Bilateral, single issue Game
'''
import random
# from negmas_negotiation.negotiators import MyNegotiator, my_utility_function
from negmas import (MappingUtilityFunction, AspirationNegotiator, Issue, SAOMechanism, ResponseType)
from typing import Optional

class NegotiationGame:
    
    def __init__(self, np_random=None):
        self.np_random = None
        
        self.life = None
        self._session_data = None
        self._session = None
        self.reset()
        
    def add_negotiators(self, negotiators: dict = None):
        '''
            Negotiators = {"my_negotiator": , "opponent_negotiator":}
            Add my negotiator and opponent negotiator
        '''
        for _ in negotiators:
            if 'my' in _:
                params = {'ufun':my_utility_function}
                self.add_my_negotiator(negotiators[_], params=params)
            else:
                self.add_opponent_negotiator(negotiators[_])
            
    
    def add_my_negotiator(self, negotiator, params: dict=None):
        self.my_negotiator = negotiator
        self.my_negotiator_params = params
        # import pdb;pdb.set_trace()
        if not params:
            self._session.add(self.my_negotiator, ufun=MappingUtilityFunction(lambda x: random.random() * x[0]))
        else:
            self._session.add(self.my_negotiator, ufun=self.my_negotiator_params['ufun'])
    
    def add_opponent_negotiator(self, negotiator, params:dict = None):
        self.opponent_negotiator = negotiator
        self.opponent_negotiator_params = params
        if not params:
            self._session.add(self.opponent_negotiator, ufun=MappingUtilityFunction(lambda x: random.random() * x[0]))
        else:
            self._session.add(self.opponent_negotiator, ufun=params['ufun'])
        
    def run(self):
        self._session.run()
    
    def step(self, action:int=None):
        '''
            Can_Improv
            Run the game session step by step. 
            Return reward, now just based on the agreement, 
            can add the time paramters
        '''
        self.action = action
        self.my_negotiator.set_current_action(self.action)
        # print('session id', self._session.id)
        # print('session time', self._session.time)
        result = self._session.step()
        result = self._session.state
        # if not self._session._running:
        #     raise StopIteration
        
        # The logic of reward
        # during classification, 
            # first condition: t<=t_end, Agreement, Ub（x, t）
            # second condtion: t<=t_end, No Deal
            # third  condition: if a_t = Counter-offer, a_t' = ResponseType.REJECT_OFFER
            # otherwise: 0
        # during regression, if the a_t is counter-offer, ResponseTyep.REJECT_OFFER in negmas
            # first condition: t<=t_end, x<= all_i belong to O_t, Ub(x, t)
            # second condtion: t<=t_end, x>all_i belong to O_t, -1
            # last condtion: otherwise, 0
        # TODO: self.my_negotiator.propose_offer
        reward = 0
        print('x best is:', self.get_x_best())
        # print(self.my_negotiator.checked_proposal, self.my_negotiator.my_last_proposal_utility)
        if self.my_negotiator.get_time()<=self.my_negotiator.get_maximum_end_time():
            # Agreement
            if self.my_negotiator.current_action == ResponseType.ACCEPT_OFFER:
                if result.agreement:
                    # if result.agreement[0] < self.my_negotiator.get_reserved_price():
                    # import pdb;pdb.set_trace()
                    print('reward agreement!', result.agreement)
                    print('rp, ip', self.my_negotiator.get_reserved_price(), self.my_negotiator.get_initial_price())
                    reward = self.my_negotiator.get_utility(result.agreement)
                    reward += 0.5 # extra reward for negotiator when get a agreement
                    # print('reward agreement!')
                    # else:
                    #     reward = -1
                else:
                    # import pdb;pdb.set_trace()

                    print('reward 0, when the offer is None, but set the action as ACCEPT OFFER!')
                    reward = 0
            # at = Counter-Offer
            elif self.my_negotiator.current_action == ResponseType.REJECT_OFFER:
                if self.my_negotiator.checked_proposal \
                            and self.my_negotiator._my_last_proposal_utility \
                                and self.my_negotiator._my_last_proposal[0] <= self.get_x_best():
                    # useful proposal
                    reward = self.my_negotiator._my_last_proposal_utility
                    print('reward reject offer new offer')
                    self.my_negotiator.checked_proposal = False
                    self.my_negotiator._my_last_proposal = None
                    self.my_negotiator._my_last_proposal_utility = None
                else:
                    # if proposal is greater than x best, then the reward is -1
                    print('reward reject offer the new offer is greater than x best or checked proposal is False: \
                                utility is less than the utility of reserved price', self.get_x_best())
                    reward = -1
                # reset all checked value and _last_proposal was already used

            elif isinstance(self.my_negotiator.current_action, ResponseType) and \
                    (self.my_negotiator.current_action == ResponseType.END_NEGOTIATION or  \
                        self.my_negotiator.current_action == ResponseType.NO_RESPONSE):
                print('reward No Deal')
                reward = -1
            else:
                print('reward time is less than max end time, but the Response is Wait')
                # Here need give a penalty, not just reward 0
                # reward = -(self.my_negotiator.get_time() / self.my_negotiator.get_maximum_end_time()) ** 0.3
                reward = 0
        else:
            # if the time is greater than t end, then the reward is zero.
            print("reward time is greater than maximum end time")
            reward = 0
        
        return reward
    
    def get_life(self):
        return self._session.running
        
    def get_issues(self):
        return self._session.issues
    
    def get_time(self):
        '''
            Wall time
        '''
        return self._session.state.time
    
    def get_relative_time(self):
        '''
            [0, 1]
        '''
        return self._session.state.relative_time
    
    def get_state(self):
        # import pdb;pdb.set_trace()
        return self._session.state
    
    def get_utility_space(self):
        '''
            TODO:
        '''
        pass
    
    def get_outcome_space(self):
        return self._session.outcomes
    
    
    def get_x_best(self):

        '''
            TODO: The best offer that opponent has offered based self._session.history
        '''
        k = [_.current_offer for _ in self.get_history() if _.current_proposer \
                    and 'opponent_negotiator' in _.current_proposer] 
        if k:
            return min(k)[0]
        else:
            return float('inf') / 1
            
    def get_history(self):
        return self._session.history
        
    def get_observation(self):
        '''
            Get observation state
            Important
            [350$, 20s, 340$, 530$] -> [X_best, time_left, IP, RP]
        '''
        x_best = self.get_x_best()
        return self.my_negotiator.get_obs(x_best)
    
    def reset(self):
        self._session = SAOMechanism(issues=[Issue((300, 550))], n_steps=50)
        negotiators = {"my_negotiator": MyNegotiator(),\
                    'opponent_negotiator': AspirationNegotiator(name="opponent_negotiator")}
        self.add_negotiators(negotiators)

#### Negotiation Environment

I have searched two projects which implement framework for researching reinforcement learning.

1. **open_spiel**, develped by deepmind, algorithm implemented with C++.
2. **gym**, develped by openai, pure python.

All of above projects are very useful when people want to research reinforcement learning.

But now i just want to realize  DRL-Negotiator.

**gym** is easier for me to combine the different reinforcement learning algorithm 
and Negotiator in Negmas simulator.

So at the end, i have selected **gym** to implement it.

I call the Environment as **AnegmasEnv**


In [ ]:
import gym
import numpy as np

from gym import spaces
from typing import Optional
from negmas import (MechanismState, ResponseType)
# from .negotiation_games import NegotiationGame
from gym.utils import seeding

class AnegmaEnv(gym.Env):
    
    def __init__(self):
        super(AnegmaEnv, self).__init__()
        self.game = None
        
                
        # Define the action space and observation space 
        self.observation_space = self._observ_space()
        self.action_space = self._action_space() # convert the mechanism state to observation space
        
        # set up the game
        self.game = NegotiationGame()
        self.mechanism_state: Optional[MechanismState] = None
    
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        # if call up the seed function, then set up the game with seed
        self.game = NegotiationGame(np_random=self.np_random)
        return [seed]
    
    def _observ_space(self):
        '''
        Observeration space of Negotiator,
        state: ("Xbest":, "Tleft":, "IP_my", "RP_my")
        X_best: best offer made by opponent,
        T_left: time left for my negotiator to reach t_end after the last action of opponent negotiator
        IP_my: intial price, minium price which my negotiator can offer at the start of negotiation
        RP_my: reserved price, maximum price which my negotiator can offer to opponent

        negotiation scenario:
        First step just consider a simple negotiation scenario, in other words, a simple negotiation settings
            Issue = {'laptop_price'}
            IP_b = [300 - 350] # the range of intial price of negotiator
            RP_b = [500 - 550] # the range of reserved price of my negotiator
            IP_s = [500 - 550] # the range of intial price of opponent, here not implemented, just use default setting of AspirationNegotiator
            RP_s = [300 - 350] # the range of reserved price of opponent, here not implemented, just use default setting of Aspirationnegotiator
            MD   = [2] # numbers of negotiators
            MR   = [1:1] # the ratio of my negotiator and opponent negotiator
            t_end = [0 - 210s]
            random sample a value from above range
            for example
            t_end = 1s,
            so the observ space is [(300 - 550), (0 - 1), (300-350), (500-550)]
        '''
        observ_space = spaces.Box(low=np.array([300, 0, 300, 500]), high=np.array([550, 1, 350, 550]), dtype=np.int)
        return observ_space
    
    
    def _action_space(self):
        
        '''
        Action Space, Discrete space.
        actions that agent can select are [0, 1, 2, 3, 4]
        0. ResponseType.REJECT_OFFER, will be gived a chance to propose new offer.
        1. ResponseType.ACCEPT_OFFER
        2. ResponseType.END_NEGOTIATION
        3. ResponseType.NO_RESPONSE, refuse to offer.
        4. ResponseType.WAIT
        
        When call function respond_ and firstly get the action ResponseType.REJECT_OFFER, then will call the function proposal_ to 
        get the new offer.
        
        The details are described in the paper: `A Deep Reinforcement learning approach to concurrent negotiation`
        '''
        
        action_space = spaces.Discrete(5)
        return action_space
    
    def step(self, action: int=None):
        '''
            Gym env, step function!
        '''
        print('\033[31m action comes from DRL is', ResponseType(action))
        print('\033[0m')
        print('n_steps is', self.n_step)
        done = False
        self.n_step += 1
        self.time = self.game.get_time()
        
        # get the reward, and run one step in game
        reward = self.game.step(action)
        # reward = 0
        
        obs = self.get_obs()
        
        if self.n_step >= self.game._session.n_steps \
            or self.time >=self.game._session.time_limit:
            done = True
        
        if not self.game.get_life():
            done = True
        
        # if self.game.my_negotiator.get_current_action() == ResponseType.END_NEGOTIATION:
        #     # if my negotiator proposed a not rational offer, then end the negotiation, 
        #     # but the negmas simulator do not know this response, so manual finished this episode.
        #     done = True
        
        # determine which info Need to be added!
        info = {
            'state': self.game.get_state()
        }
        # print(obs, reward, done, info)
        print(reward)
        return obs, reward, done, info
    
    def init_game_state(self):
        self.n_step = 0
        self.time = 0
        self.game.reset()
    
    def get_obs(self):
        '''
            Get obs state
        '''
        return self.game.get_observation()
    
    def reset(self):
        '''
            Reset the state of the environment to an initial state,
            When the value of done from the result of function step() is True, 
            this function will be called
            create a new game
        '''
        # print("reset has been called")
        self.init_game_state()
        # print(id(self.game), id(self.game._session))
        return self.get_obs() # the initial observation space
    
    def render(self, mode='human', close=False):
        '''
            Render the environment to the screen
        '''
        pass
    
    def close(self):
        pass
    


### Example Train with DQN

In [ ]:
import os
import gym

import matplotlib.pyplot as plt 

from stable_baselines import DQN
from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines import results_plotter
from stable_baselines.bench import Monitor

NUM_TIMESTEPS = int(1000)
# The seed for generating the random number
SEED = 721
# Setting for Evaluation
EVAL_FREQ = 100
EVAL_EPISODES = 2
# LOGDIR 
LOGDIR = "Test_dqn"

logger.configure(folder=LOGDIR)

env=AnegmaEnv()
env = Monitor(env, LOGDIR)
env.seed(SEED)
# import pdb;pdb.set_trace()
# model = PPO1(MlpPolicy, env, timesteps_per_actorbatch=40, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
#                  optim_stepsize=3e-4, optim_batchsize=2, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

model = DQN('MlpPolicy', env, learning_rate=1e-3, prioritized_replay=True, verbose=1)
eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

model.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

model.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()


### load and show the result

In [ ]:
NUM_TIMESTEPS = int(1000)
LOGDIR = "Test_dqn"
results_plotter.plot_results([LOGDIR], NUM_TIMESTEPS, results_plotter.X_TIMESTEPS, "DQN Negotiation")
plt.show()

### Evaluate the result that i have runned 5000000 steps

In [ ]:
# ATTENTION: 
# If you run by yourself, will take a moment
# or can just skip this cell, see the result i have shown in next cell

from stable_baselines import results_plotter
import matplotlib.pyplot as plt

NUM_TIMESTEPS = 5000000
LOGDIR = "dqn"
results_plotter.plot_results([LOGDIR], NUM_TIMESTEPS, results_plotter.X_TIMESTEPS, "DQN Negotiation")
plt.savefig('dqn_result.png', dpi=600)

#### Result of 5000000 steps

The blue circle: monitor.csv, the sum reward of every episode: means one finished negotiation session.

The blue line:  progress.csv, mean 100 episode reward.

From the figure, to a certain extent, the DRL-Negotiator learn a strategy 
to reponse the opponent based on the state that i have defined in source code.

Problem:

1. spent too much time to learn, only after 2000000 steps, could see the slight change.
2. oscillation: Reward suddenly went very low 

![result after 5000000 steps](./dqn_result.png)

### Questions and future Work in 2020/08 - 2020/09

Future work

1. Problem in Result of 5000000 steps that i have mentioned, I will try to use another DRL algorithm, for example, trop, ppo and so on, at the end compare the result.
2. Use multi processing to learn, accelerate learn
3. Change and refine the observation space, action space, reward
4. Different negotiation settings, different negotiators and so on.
5. Make the code more robust and suitable for more scenarios: There are too many fixed parameters now  

Questions:

1. I have seen too few papers using DRL to learn how to negotiate. If you have, could you send to me? It will be very useful.
2. Whether I need to analyze the scenario: multi issue, large scale numbers of negotiators, if it is necessarily, could you give some suggestions?
3. Can simply transfer this negotiator to scml? I will check it.

**Important Questions:** 
How to combine Acceptance Strategy and Bidding Strategy?

When i use DRL to learn how to decide the next action, i have found an interesting situation

1. Just consider the response action as ResponseType.ACCEPT_OFFER, ResponseType.REJECT_OFFER... here is simple,just set the action as discrete value.(Acceptance Strategy)
2. But when the action is ResponseType.REJECT_OFFER, it is possible that my negotiator create a new offer. In this situation, how to decide the outcome is a important question.(Bidding Strategy)

in machine learning, propose an outcome is a regression Problem, response is a classification problem. 

So my Questions is, if it is possibe, i create a new model, the input is the negotiation state, first solve the classification problem(response), if the action is ResponseType.REJECT_OFFER, then at the same time, this DRL model output an outcome (whether it is considered as continous action space?).

--------------------------------------------

If you have any suggestions, you can send to me, I need all of them.

I need more definite direction and goals. 
It may be better to determine the title of the paper. 

I feel more and more interesting in this direction and also find many problems in this field.